群里那个**wifi_baseline.py**是我第一次提交时候的代码，直接读的话会比较吃力，特意写了这个说明文档，顺带介绍下规则做法还有一开始的思路

In [1]:
import pandas as pd
from collections import defaultdict

先是读取文件并大概看下数据长啥样

In [2]:
user_shop_hehavior = pd.read_csv('ccf_first_round_user_shop_behavior.csv')
user_shop_hehavior.head()

,user_id,shop_id,time_stamp,longitude,latitude,wifi_infos
0,u_376,s_2871718,2017-08-06 21:20,122.308291,32.088040,b_6396480|-67|false;b_41124514|-86|false;b_287...
1,u_376,s_2871718,2017-08-06 21:20,122.308162,32.087970,b_6396480|-67|false;b_56328155|-73|false;b_411...
2,u_1041,s_181637,2017-08-02 13:10,117.365255,40.638214,b_8006367|-78|false;b_2485110|-52|false;b_3005...
3,u_1158,s_609470,2017-08-13 12:30,121.134451,31.197416,b_26250579|-73|false;b_26250580|-64|false;b_26...
4,u_1654,s_3816766,2017-08-25 19:50,122.255867,31.351320,b_39004150|-66|false;b_39004148|-58|false;b_21...


In [3]:
evalution = pd.read_csv('evaluation_public.csv')
evalution.head()

,row_id,user_id,mall_id,time_stamp,longitude,latitude,wifi_infos
0,118742,u_30097142,m_3916,2017-09-05 13:00,122.141011,39.818847,b_34366982|-82|false;b_37756289|-53|false;b_41...
1,118743,u_30097803,m_5085,2017-09-06 13:10,118.191907,32.855858,b_36722251|-81|false;b_10537579|-75|false;b_43...
2,118744,u_30097889,m_4033,2017-09-06 17:40,119.192110,32.424667,b_30026291|-74|false;b_30026290|-74|false;b_36...
3,118745,u_30098996,m_4515,2017-09-03 12:10,120.612201,34.055249,b_33412374|-77|false;b_22084893|-86|false;b_52...
4,118746,u_30099170,m_7168,2017-09-02 20:40,116.861989,40.326858,b_19882704|-77|false;b_2241462|-49|false;b_585...


看了下数据说明，打算先从wifi入手，于是就有了一个想法，**同一家店周围的wifi应该差不多**

接下来是想法转换成规则

就数据来看，需要把**wifi_infos**转化为**shop_id**

## 实现规则

思考一下：凭什么看见这些wifi就能推断出是这家店

**因为逛这家店的周围总是这些wifi**

有了这个想法就简单了

那就统计每个wifi和每个shop的关联数吧

In [4]:
wifi_to_shops = defaultdict(lambda : defaultdict(lambda :0)) # 默认字典嵌套  wifi_to_shops[wifi][shop]为wifi与shop的关联个数
for line in user_shop_hehavior.values:
    wifi = [wifi.split('|') for wifi in line[5].split(';')]
    for each_wifi in wifi:
        wifi_to_shops[each_wifi[0]][line[1]] += 1 # each_wifi[0] 表示wifi的bssid  line[1]表示shop_id

有了这个小框架就能添想法了

比如这个想法：有的wifi信号太差，别的不一定收得到，那就干脆不要了

代码实现：那就添加个topK

In [5]:
topK = 5
wifi_to_shops = defaultdict(lambda : defaultdict(lambda :0)) # 默认字典嵌套  wifi_to_shops[wifi][shop]为wifi与shop的关联个数
for line in user_shop_hehavior.values:
    wifi = sorted([wifi.split('|') for wifi in line[5].split(';')],key=lambda x:int(x[1]),reverse=True)[:topK]
    for each_wifi in wifi:
        wifi_to_shops[each_wifi[0]][line[1]] += 1 # each_wifi[0] 表示wifi的bssid  line[1]表示shop_id

想法：信号强的wifi肯定重要点

代码：加权统计

In [6]:
topK_weights = [5,4,3,2,1]
wifi_to_shops = defaultdict(lambda : defaultdict(lambda :0)) # 默认字典嵌套  wifi_to_shops[wifi][shop]为wifi与shop的关联个数
for line in user_shop_hehavior.values:
    wifi = sorted([wifi.split('|') for wifi in line[5].split(';')],key=lambda x:int(x[1]),reverse=True)[:len(topK_weights)]
    for i,each_wifi in enumerate(wifi):
        wifi_to_shops[each_wifi[0]][line[1]] += topK_weights[i] # each_wifi[0] 表示wifi的bssid  line[1]表示shop_id

想法：连接着的wifi虽然不一定有别的信号强，但也挺重要的

代码：单独设权

In [7]:
topK_weights = [5,4,3,2,1]
connect_weight = 5
wifi_to_shops = defaultdict(lambda : defaultdict(lambda :0)) # 默认字典嵌套  wifi_to_shops[wifi][shop]为wifi与shop的关联个数
for line in user_shop_hehavior.values:
    wifi = sorted([wifi.split('|') for wifi in line[5].split(';')],key=lambda x:int(x[1]),reverse=True)[:len(topK_weights)]
    for i,each_wifi in enumerate(wifi):
        if each_wifi[2] == 'true':
            wifi_to_shops[each_wifi[0]][line[1]] += connect_weight # 单独设权
        else:
            wifi_to_shops[each_wifi[0]][line[1]] += topK_weights[i] # each_wifi[0] 表示wifi的bssid  line[1]表示shop_id

随便挑一个wifi来看看

In [8]:
wifi_to_shops['b_6396480']

defaultdict(<function __main__.<lambda>.<locals>.<lambda>>,
            {'s_1877554': 6, 's_2871718': 132, 's_580441': 9})

## 线下尝试

规则做线下，一方面可以检测规则是否写错，另一方面可以很好地调整参数

对于上面做好的wifi_to_shops，怎么根据周围的wifi来预测shop呢？

**那就加起来呗，谁分高选谁**

和上面的思路差不多，也是整个topK

In [9]:
right_count = 0
result_topK = 5 #根据周围最强的k个wifi来预测
for line in user_shop_hehavior.values:
    wifi = sorted([wifi.split('|') for wifi in line[5].split(';')],key=lambda x:int(x[1]),reverse=True)[:result_topK]
    counter = defaultdict(lambda : 0) #统计每家店的得分
    for each_wifi in wifi:
        for k,v in wifi_to_shops[each_wifi[0]].items():
            counter[k] += v
    pred_one = sorted(counter.items(),key=lambda x:x[1],reverse=True)[0][0]
    if pred_one == line[1]:
        right_count += 1
print('acc:',1.0*right_count/len(user_shop_hehavior))

acc: 0.7634486364415232


## 暴力调参

规则比较有趣的地方就是你的思路不用先想好具体的值，只要定好思路，然后让电脑试就行了

In [11]:
topK_weights_list = [[5,4,3,2,1],[3,2,1],[1]]
for topK_weights in topK_weights_list:
    for connect_weight in [topK_weights[0],topK_weights[0]*2,topK_weights[0]/2.0]:
        
        #上面小框架搭好就能直接扔过来用了
        
        #生成规则
        wifi_to_shops = defaultdict(lambda : defaultdict(lambda :0)) # 默认字典嵌套  wifi_to_shops[wifi][shop]为wifi与shop的关联个数
        for line in user_shop_hehavior.values:
            wifi = sorted([wifi.split('|') for wifi in line[5].split(';')],key=lambda x:int(x[1]),reverse=True)[:len(topK_weights)]
            for i,each_wifi in enumerate(wifi):
                if each_wifi[2] == 'true':
                    wifi_to_shops[each_wifi[0]][line[1]] += connect_weight # 单独设权
                else:
                    wifi_to_shops[each_wifi[0]][line[1]] += topK_weights[i] # each_wifi[0] 表示wifi的bssid  line[1]表示shop_id

        #线下测试
        result_topKs = [1,len(topK_weights),int(len(topK_weights)/2),10]
        for result_topK in result_topKs: #根据周围最强的k个wifi来预测
            right_count = 0
            for line in user_shop_hehavior.values:
                wifi = sorted([wifi.split('|') for wifi in line[5].split(';')],key=lambda x:int(x[1]),reverse=True)[:result_topK]
                counter = defaultdict(lambda : 0) #统计每家店的得分
                for each_wifi in wifi:
                    for k,v in wifi_to_shops[each_wifi[0]].items():
                        counter[k] += v
                pred_one = sorted(counter.items(),key=lambda x:x[1],reverse=True)[0][0]
                if pred_one == line[1]:
                    right_count += 1
            print(topK_weights,connect_weight,result_topK,'acc:',1.0*right_count/len(user_shop_hehavior)) #算好一个输出一个 等着也算不无聊

[5, 4, 3, 2, 1] 5 1 acc: 0.8371339569337838
[5, 4, 3, 2, 1] 5 5 acc: 0.7634486364415232
[5, 4, 3, 2, 1] 5 2 acc: 0.8163609442757784
[5, 4, 3, 2, 1] 5 10 acc: 0.6895656032653348
[5, 4, 3, 2, 1] 10 1 acc: 0.8391198710034578
[5, 4, 3, 2, 1] 10 5 acc: 0.7648598656432473
[5, 4, 3, 2, 1] 10 2 acc: 0.8179962478526206
[5, 4, 3, 2, 1] 10 10 acc: 0.6905506517928147
[5, 4, 3, 2, 1] 2.5 1 acc: 0.8350619280062214
[5, 4, 3, 2, 1] 2.5 5 acc: 0.7625470666028128
[5, 4, 3, 2, 1] 2.5 2 acc: 0.8149743193191654
[5, 4, 3, 2, 1] 2.5 10 acc: 0.688580554737855
[3, 2, 1] 3 1 acc: 0.8546152730851526
[3, 2, 1] 3 3 acc: 0.8104655914025738
[3, 2, 1] 3 1 acc: 0.8546152730851526
[3, 2, 1] 3 10 acc: 0.6885972504756088
[3, 2, 1] 6 1 acc: 0.855538810999855
[3, 2, 1] 6 3 acc: 0.8113135591358638
[3, 2, 1] 6 1 acc: 0.855538810999855
[3, 2, 1] 6 10 acc: 0.6895190309442318
[3, 2, 1] 1.5 1 acc: 0.8531996502682302
[3, 2, 1] 1.5 3 acc: 0.8093979429093641
[3, 2, 1] 1.5 1 acc: 0.8531996502682302
[3, 2, 1] 1.5 10 acc: 0.6872686212

IndexError: list index out of range

报错了，查看一下，不是啥大问题，反正参数也跑好了，最佳规则是[1] 1 1

In [13]:
topK_weights,connect_weight,result_topK

([1], 1, 0)

改进后的代码如下，再跑挺耗时，就这么放着吧

In [ ]:
topK_weights_list = [[5,4,3,2,1],[3,2,1],[1]]
for topK_weights in topK_weights_list:
    for connect_weight in [topK_weights[0],topK_weights[0]*2,topK_weights[0]/2.0]:
        
        #上面小框架搭好就能直接扔过来用了
        
        #生成规则
        wifi_to_shops = defaultdict(lambda : defaultdict(lambda :0)) # 默认字典嵌套  wifi_to_shops[wifi][shop]为wifi与shop的关联个数
        for line in user_shop_hehavior.values:
            wifi = sorted([wifi.split('|') for wifi in line[5].split(';')],key=lambda x:int(x[1]),reverse=True)[:len(topK_weights)]
            for i,each_wifi in enumerate(wifi):
                if each_wifi[2] == 'true':
                    wifi_to_shops[each_wifi[0]][line[1]] += connect_weight # 单独设权
                else:
                    wifi_to_shops[each_wifi[0]][line[1]] += topK_weights[i] # each_wifi[0] 表示wifi的bssid  line[1]表示shop_id

        #线下测试
        result_topKs = [1,len(topK_weights),int(len(topK_weights)/2),10]
        for result_topK in result_topKs: #根据周围最强的k个wifi来预测
            if result_topK == 0:
                break
            right_count = 0
            for line in user_shop_hehavior.values:
                wifi = sorted([wifi.split('|') for wifi in line[5].split(';')],key=lambda x:int(x[1]),reverse=True)[:result_topK]
                counter = defaultdict(lambda : 0) #统计每家店的得分
                for each_wifi in wifi:
                    for k,v in wifi_to_shops[each_wifi[0]].items():
                        counter[k] += v
                try:
                    pred_one = sorted(counter.items(),key=lambda x:x[1],reverse=True)[0][0]
                except:
                    pred_one = None
                if pred_one == line[1]:
                    right_count += 1
            print(topK_weights,connect_weight,result_topK,'acc:',1.0*right_count/len(user_shop_hehavior)) #算好一个输出一个 等着也算不无聊

## 最终规则代码

根据暴力跑出来的参数，得到最佳规则——**选信号最强的一个来统计，再根据用户周围最强的来找统计次数最多的商店**

经过一点点优化，**wifi_baseline.py**就这么出来了